In [83]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, List, NotRequired
import random

In [84]:

class AgentState(TypedDict):
    player_name: str
    guesses: List[int]
    attempts: int
    lower_bound: int
    upper_bound: int
    target: NotRequired[int]

In [85]:
def setup_game(state: AgentState) -> AgentState:
    """Setup the number guessing game with a random target"""
    target = random.randint(1, 20)
    print(f"🎮 Game started for {state['player_name']}! Guess a number between 1 and 20.")
    return {
        **state,
        "target": target
    }


In [86]:
def make_guess(state: AgentState) -> AgentState:
    """Make a smart guess within the current bounds"""
    lower = state["lower_bound"]
    upper = state["upper_bound"]
    
    # Make a smart guess (middle of the range)
    guess = (lower + upper) // 2
    
    print(f"🤔 Guessing: {guess} (range: {lower}-{upper})")
    
    return {
        **state,
        "attempts": state["attempts"] + 1,
        "guesses": state["guesses"] + [guess]
    }


In [87]:
def provide_hint(state: AgentState) -> AgentState:
    """Provide hint after a guess and update bounds accordingly"""
    last_guess = state["guesses"][-1]
    target = state.get("target", 5)
    
    if last_guess < target:
        print(f"📈 Too low! Try higher than {last_guess}")
        return {
            **state,
            "lower_bound": last_guess + 1
        }
    elif last_guess > target:
        print(f"📉 Too high! Try lower than {last_guess}")
        return {
            **state,
            "upper_bound": last_guess - 1
        }
    else:
        print(f"🎉 Correct! The number was {target}")
        return state


In [88]:
def should_continue(state: AgentState) -> str:
    """Decide whether to continue guessing or end the game"""
    
    last_guess = state["guesses"][-1]
    target = state.get("target", 5)
    max_attempts = 7
    
    # Max attempts reached 
    if state["attempts"] >= max_attempts:
        if last_guess == target:
            print(f"🎉 Correct! The number was {target}")
            print(f"💀 But you used all {max_attempts} attempts! Game over!")
        else:
            print(f"💀 Game over! The number was {target}. Better luck next time!")
        return "end"
    
    # Win condition
    if last_guess == target:
        print(f"🏆 {state['player_name']} won in {state['attempts']} attempts!")
        return "end"
    
    # Continue guessing
    return "continue"


In [89]:
# Build the graph  
graph = StateGraph(AgentState)

# Add nodes
graph.add_node("setup", setup_game)
graph.add_node("guess", make_guess)
graph.add_node("hint", provide_hint)

# Add edges
graph.add_edge(START, "setup")
graph.add_edge("setup", "guess")
graph.add_edge("guess", "hint")

# Add conditional edge
graph.add_conditional_edges(
    "hint",
    should_continue,
    {
        "continue": "guess",
        "end": END
    }
)

# Compile the graph
app = graph.compile()


In [90]:
result = app.invoke({
    "player_name": "Student", 
    "guesses": [],
    "attempts": 0,
    "lower_bound": 1,
    "upper_bound": 20
})
print(f"\n🎯 Final result: {result}")


🎮 Game started for Student! Guess a number between 1 and 20.
🤔 Guessing: 10 (range: 1-20)
📉 Too high! Try lower than 10
🤔 Guessing: 5 (range: 1-9)
📈 Too low! Try higher than 5
🤔 Guessing: 7 (range: 6-9)
📈 Too low! Try higher than 7
🤔 Guessing: 8 (range: 8-9)
📈 Too low! Try higher than 8
🤔 Guessing: 9 (range: 9-9)
🎉 Correct! The number was 9
🏆 Student won in 5 attempts!

🎯 Final result: {'player_name': 'Student', 'guesses': [10, 5, 7, 8, 9], 'attempts': 5, 'lower_bound': 9, 'upper_bound': 9, 'target': 9}
